In [1]:
import boto3
from sagemaker.model import Model
from sagemaker import get_execution_role, Session

role = get_execution_role()
session = Session()

ecr_image = '491085388405.dkr.ecr.us-east-1.amazonaws.com/fraud-ecr-27:latest'

model = Model(
    image_uri=ecr_image,
    role=role,
    sagemaker_session=session
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name='custom-ml-endpoint'
)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
----!

In [2]:
!pip install psycopg2-binary boto3 sqlalchemy

In [3]:
!pip install supabase

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached pydantic_core-2.33.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached typing_inspection-0.4.1-py3-none-any.whl (14 kB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:
      Successfully uninstalled pydantic_core-2.23.4
  Attempting uninstall: pydantic90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/14 [pydantic-core]
    Found existing install

In [4]:
###### Fetching the Data from Supabase and storing in S3 ########
#################################################################
# print("Supabase raw response:", response)
# print("Supabase data:", response.data)


# print("DataFrame shape:", df.shape)
# print("DataFrame head:\n", df.head())


from supabase import create_client, Client
import pandas as pd
import boto3

# Supabase config
SUPABASE_URL = "https://mcgzvjzuqnjstptfrsuj.supabase.co"
SUPABASE_KEY = "sb_publishable_4vx6B3HjwnEQVwI9SgrJkQ_S32tu5l5"

# Use the service_role key, not anon

# Create Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Fetch data
response = supabase.table("loan_fraud_analytics").select("*").execute()
df = pd.DataFrame(response.data)

# Save as CSV
csv_path = "/tmp/loan_fraud_data.csv"
df.to_csv(csv_path, index=False)

# Upload to S3
s3 = boto3.client("s3")
bucket_name = "manas-bucket100"
object_key = "input-file/loan_fraud_data.csv"

s3.upload_file(csv_path, bucket_name, object_key)
print("✅ Upload to S3 complete")


✅ Upload to S3 complete


In [5]:
######## Dropping the target variable ########
##############################################
import boto3
import io

# Drop target column
df_features = df.drop(columns=['loan_default'])

# Convert to CSV buffer
csv_buffer = io.StringIO()
df_features.to_csv(csv_buffer, index=False)

# Upload to S3
s3 = boto3.client('s3')
bucket_name = 'manas-bucket100'  # 🔁 Your bucket name
object_key = 'input-file/loan_fraud_data.csv'

s3.put_object(Bucket=bucket_name, Key=object_key, Body=csv_buffer.getvalue())

print(f"✅ Features uploaded to s3://{bucket_name}/{object_key}")


✅ Features uploaded to s3://manas-bucket100/input-file/loan_fraud_data.csv


In [6]:
####### Code for Data processing ########
import pandas as pd
import numpy as np
import os
import boto3
from io import StringIO
bucket_name = "manas-bucket100"
s3_key = "input-file/loan_fraud_data.csv"

# Step 1: Define S3 download
def load_csv_from_s3(bucket_name, file_key):
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket=bucket_name, Key=file_key)
    content = response['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(content))
    print(f"✅ Loaded data from s3://{bucket_name}/{file_key}")
    return df

# Step 2: Handle missing values
def handle_missing_values(df):
    df = df.copy()
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col].fillna(df[col].mode()[0], inplace=True)
        else:
            df[col].fillna(df[col].median(), inplace=True)
    return df

# Step 3: Handle outliers
def handle_outliers(df):
    df = df.copy()
    for col in df.select_dtypes(include=np.number).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df[col] = np.where(df[col] < lower, lower,
                  np.where(df[col] > upper, upper, df[col]))
    return df

# Step 4: Create dummies
def create_dummies(df):
    df = pd.get_dummies(df, drop_first=True, dtype=int)
    return df

# Step 5: Drop low-variance and ID-like columns
def drop_low_variance_and_id_columns(df, threshold=0.95):
    df = df.copy()
    drop_cols = []

    for col in df.columns:
        if df[col].nunique() <= 1:
            drop_cols.append(col)
        else:
            top_freq_ratio = df[col].value_counts(normalize=True).values[0]
            if top_freq_ratio >= threshold:
                drop_cols.append(col)

    id_like_cols = [col for col in df.columns if col.lower() == 'id'
                    or col.lower().startswith('id')
                    or col.lower().endswith('id')
                    or '_id' in col.lower()
                    or 'id_' in col.lower()]
    
    drop_cols = list(set(drop_cols + id_like_cols))

    if drop_cols:
        print(f"🧹 Dropping low-variance/id-like columns: {drop_cols}")
        df.drop(columns=drop_cols, inplace=True)

    return df

# Step 6: Run all preprocessing
def preprocess_data(df):
    df = drop_low_variance_and_id_columns(df, threshold=0.95)
    df = handle_missing_values(df)
    df = handle_outliers(df)
    df = create_dummies(df)
    return df

# MAIN execution inside SageMaker
def main():
    bucket_name = 'manas-bucket100'
    file_key = 'input-file/loan_fraud_data.csv'  # full path in S3 bucket

    df = load_csv_from_s3(bucket_name, file_key)
    print(f"📥 Original shape: {df.shape}")

    df = preprocess_data(df)
    print(f"📊 Processed shape: {df.shape}")

    # Save to local file or upload to S3 if needed
    processed_file = "processed_test_data.csv"
    df.to_csv(processed_file, index=False)
    print(f"✅ Preprocessed test data saved: {processed_file}")

    return df

# Run if in script mode
if __name__ == "__main__":
    main()


✅ Loaded data from s3://manas-bucket100/input-file/loan_fraud_data.csv
📥 Original shape: (1000, 9)
🧹 Dropping low-variance/id-like columns: ['last_delinq_none', 'loan_id']
📊 Processed shape: (1000, 10)
✅ Preprocessed test data saved: processed_test_data.csv


/tmp/ipykernel_15001/2229654881.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipykernel_15001/2229654881.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [12]:
import boto3
import json

# Create SageMaker runtime client
client = boto3.client('sagemaker-runtime', region_name='us-east-1')  # update region if needed

# Sample input — should match model's expected feature shape and order
payload = {
    "inputs": [[27.0, 1.0, 500987.0, 820870.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0]]
}

# Invoke the deployed endpoint
response = client.invoke_endpoint(
    EndpointName='custom-ml-endpoint',  # Make sure this matches exactly
    Body=json.dumps(payload),
    ContentType='application/json'
)

# Read and decode the prediction
result = response['Body'].read().decode('utf-8')
print("Prediction:", result)

Prediction: {"predictions":[0]}



In [28]:
import boto3
import pandas as pd
import json

# Load your CSV
df = pd.read_csv('processed_test_data.csv')  # Each row should have feature values

# Convert the data to list of lists (assuming features are in columns)
data = df.values.tolist()

# Initialize SageMaker client
client = boto3.client('sagemaker-runtime')

# Predict in a loop (or you can do it in batches)
predictions = []
for row in data:
    payload = json.dumps({'inputs': [row]})
    response = client.invoke_endpoint(
        EndpointName='custom-ml-endpoint',
        Body=payload,
        ContentType='application/json'
    )
    result = json.loads(response['Body'].read().decode()) 
    predictions.append(result)

# Save predictions to CSV
df['prediction_fraudulent'] = predictions
df.to_csv('predictions_fraudulent.csv', index=False)


In [ ]:
####### Fetching the inference log from Sagemaker endpoint and saving it in S3 and locally########
##################################################################################################
import boto3
import pandas as pd 
import json
import uuid
from datetime import datetime

# --- Config ---
endpoint_name = 'custom-ml-endpoint'
s3_bucket = 'manas-bucket100'
s3_key = f'logs/inference_logs_{datetime.utcnow().strftime("%Y-%m-%d_%H-%M-%S")}.csv'

# --- Load data ---
df = pd.read_csv('processed_test_data.csv')  # Each row should have feature values
data = df.values.tolist()
columns = df.columns.tolist()

# --- Initialize clients ---
client = boto3.client('sagemaker-runtime')
s3 = boto3.client('s3')

# --- Prediction + Logging ---
log_rows = []
predictions = []

for row in data:
    payload = json.dumps({'inputs': [row]})
    try:
        response = client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=payload,
            ContentType='application/json'
        )
        result = json.loads(response['Body'].read().decode())
        prediction = result["predictions"][0] if "predictions" in result else result
    except Exception as e:
        prediction = None
        print(f"Prediction error: {e}")

    log_entry = {
        'uuid': str(uuid.uuid4()),
        'timestamp': datetime.utcnow().isoformat(),
        **{f: v for f, v in zip(columns, row)},
        'prediction': prediction
    }

    log_rows.append(log_entry)
    predictions.append(prediction)

# --- Save to CSV ---
log_df = pd.DataFrame(log_rows)
log_df.to_csv('inference_logs.csv', index=False)
print("✅ Inference logs saved locally as 'inference_logs.csv'")

# --- Upload to S3 ---
with open("inference_logs.csv", "rb") as f:
    s3.upload_fileobj(f, Bucket=s3_bucket, Key=s3_key)
    print(f"✅ Log file uploaded to s3://{s3_bucket}/{s3_key}")


✅ Inference logs saved locally as 'inference_logs.csv'
✅ Log file uploaded to s3://manas-bucket100/logs/inference_logs_2025-07-27_10-03-12.csv


In [30]:
###### Drift Detections ########
################################
import pandas as pd
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt

# --- Load Data ---
reference_df = pd.read_csv("processed_test_data.csv")  # Your stored baseline (used for training)
inference_df = pd.read_csv("inference_logs.csv")  # Today's logged inputs

# --- Feature Columns (exclude prediction column) ---
feature_columns = [col for col in reference_df.columns if col in inference_df.columns and col not in ['prediction', 'timestamp', 'uuid']]

# --- Run KS-Test and Plot ---
drift_results = []

for col in feature_columns:
    ref_values = reference_df[col].dropna()
    inf_values = inference_df[col].dropna()

    # Kolmogorov-Smirnov test
    statistic, p_value = ks_2samp(ref_values, inf_values)

    drift_results.append({
        'feature': col,
        'ks_statistic': statistic,
        'p_value': p_value,
        'drift_detected': p_value < 0.05
    })

    # Optional: Plot distribution
    plt.figure(figsize=(6, 3))
    plt.hist(ref_values, bins=30, alpha=0.5, label='Reference')
    plt.hist(inf_values, bins=30, alpha=0.5, label='Inference')
    plt.title(f'Distribution of {col}')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'drift_plot_{col}.png')
    plt.close()

# --- Save Results ---
drift_df = pd.DataFrame(drift_results)
drift_df.to_csv('drift_detection_report.csv', index=False)
print("✅ Drift detection report saved as 'drift_detection_report.csv'")


Matplotlib is building the font cache; this may take a moment.


✅ Drift detection report saved as 'drift_detection_report.csv'
